<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_2_1_Replicating_Gravitational_Lensing_(Delayed_Light).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Simulation 2.1: NGFT Gravitational Lensing & Time Delay
# Language: Julia (V6 - Final Tuning)

using Plots
using Printf
using LinearAlgebra # For norm()

println("--- NGFT Gravitational Lensing Simulation (V6 - Final Tuning) ---")
println("Phase 1: Running physics calculations...")

# --- 1. Simulation Parameters (Tuned & const removed) ---
# Removed 'const' to prevent redefinition warnings in interactive environments
LATTICE_SIZE = 200
TOTAL_FRAMES = 1000
DT = 0.5

# !--- KEY TUNING PARAMETERS ---!
SPEED_OF_LIGHT = 1.0                    # Giving packets a higher initial speed to escape capture
LENSING_STRENGTH = 0.25               # Significantly reduced gravity to ensure deflection, not capture
INFORMATIONAL_DRAG_COEFFICIENT = 0.04
ENTROPY_GROWTH_RATE = 0.001
VACUUM_NOISE_STRENGTH = 0.1

# !--- Scene objects tuned for a grazing encounter ---!
SOURCE_POS = [5.0, 100.0]               # Moved source further back
OBSERVER_POS = [195.0, 100.0]             # Moved observer further back
LENSING_MASS_POS = [100.0, 100.0]
LENSING_MASS_SIGMA = 15.0
OBSERVER_SIGMA = 5.0
OBSERVER_DETECT_RADIUS = 5.0

# --- 2. Data Structures & 3. Helper Functions (Unchanged) ---
mutable struct LightPacket
    pos::Vector{Float64}
    vel::Vector{Float64}
    is_active::Bool
    travel_time::Float64
    detection_frame::Int
end

function generate_gaussian(size, center, sigma)
    x = [i for i in 1:size, j in 1:size]
    y = [j for i in 1:size, j in 1:size]
    return exp.(-((x .- center[1]).^2 .+ (y .- center[2]).^2) ./ (2 * sigma^2))
end

function get_gradient(field, x, y)
    ix, iy = round(Int, x), round(Int, y)
    if ix < 2 || ix > size(field, 1) - 1 || iy < 2 || iy > size(field, 2) - 1
        return [0.0, 0.0]
    end
    grad_x = (field[ix+1, iy] - field[ix-1, iy]) / 2.0
    grad_y = (field[ix, iy+1] - field[ix, iy-1]) / 2.0
    return [grad_x, grad_y]
end

# --- 4. PHASE 1: Run Simulation (Velocity Corrected) ---

g_potential = -1.0 .* generate_gaussian(LATTICE_SIZE, LENSING_MASS_POS, LENSING_MASS_SIGMA)
g_potential .+= -0.4 .* generate_gaussian(LATTICE_SIZE, OBSERVER_POS, OBSERVER_SIGMA)
entropy_field = randn(LATTICE_SIZE, LATTICE_SIZE) .* VACUUM_NOISE_STRENGTH

# !--- Packets now start with the defined SPEED_OF_LIGHT ---!
packets = [LightPacket(copy(SOURCE_POS), [cos(a) * SPEED_OF_LIGHT, sin(a) * SPEED_OF_LIGHT], true, 0.0, -1) for a in range(-0.4, 0.4, length=12)]

path_history = [[copy(p.pos)] for p in packets]
entropy_history = zeros(TOTAL_FRAMES, LATTICE_SIZE, LATTICE_SIZE)

# Main calculation loop remains the same as V4
for frame in 1:TOTAL_FRAMES
    global entropy_field .+= (ENTROPY_GROWTH_RATE * DT) .+ (randn(LATTICE_SIZE, LATTICE_SIZE) .* VACUUM_NOISE_STRENGTH)
    entropy_history[frame, :, :] = entropy_field
    for i in 1:length(packets)
        if packets[i].is_active
            grad = get_gradient(g_potential, packets[i].pos[1], packets[i].pos[2])
            accel = -LENSING_STRENGTH * grad
            packets[i].vel += accel * DT

            ix, iy = round(Int, packets[i].pos[1]), round(Int, packets[i].pos[2])
            drag_factor = 0.0
            if 1 <= ix <= LATTICE_SIZE && 1 <= iy <= LATTICE_SIZE
                drag_factor = clamp(INFORMATIONAL_DRAG_COEFFICIENT * entropy_field[ix, iy], 0.0, 0.9)
            end

            # Normalize and then apply the local speed
            current_speed = SPEED_OF_LIGHT * (1.0 - drag_factor)
            packets[i].vel = normalize(packets[i].vel) * current_speed
            packets[i].pos += packets[i].vel * DT
            packets[i].travel_time += DT

            push!(path_history[i], copy(packets[i].pos))

            if norm(packets[i].pos - OBSERVER_POS) < OBSERVER_DETECT_RADIUS
                packets[i].is_active = false
                packets[i].detection_frame = frame
            elseif any(x -> x < 0 || x > LATTICE_SIZE, packets[i].pos) # Use 0 for boundary check
                packets[i].is_active = false
            end
        end
    end
end

println("Phase 1 complete.")
println("Phase 2: Rendering animation...")

# --- 5. PHASE 2: Render Animation & 6. PHASE 3: Final Analysis (Unchanged) ---
# This code is now robust and will produce the correct final output.
anim = @animate for frame in 1:2:TOTAL_FRAMES
    p_layout = @layout [a{0.7w} b]
    p = plot(layout=p_layout, size=(1600, 800), background_color=:black)
    plot!(p[1], xlims=(0, LATTICE_SIZE), ylims=(0, LATTICE_SIZE), grid=false, aspect_ratio=:equal, title="NGFT Lensing (Frame $frame/$TOTAL_FRAMES)")
    heatmap!(p[1], 1:LATTICE_SIZE, 1:LATTICE_SIZE, g_potential', c=:thermal, colorbar=false)
    scatter!(p[1], [SOURCE_POS[1]], [SOURCE_POS[2]], marker=:star, color=:cyan, markersize=8, label="Source")
    scatter!(p[1], [OBSERVER_POS[1]], [OBSERVER_POS[2]], marker=:cross, color=:white, markersize=8, label="Observer")
    for (i, path) in enumerate(path_history)
        if !isempty(path)
            frames_in_path = length(path)
            draw_up_to = min(frame + 1, frames_in_path)
            path_xs = [pt[1] for pt in path[1:draw_up_to]]
            path_ys = [pt[2] for pt in path[1:draw_up_to]]
            plot!(p[1], path_xs, path_ys, color=:cyan, lw=1.5, label="")
            if packets[i].is_active || packets[i].detection_frame >= frame
                 scatter!(p[1], [path_xs[end]], [path_ys[end]], color=:white, markersize=2, label="")
            end
        end
    end
    plot!(p[2], title="Entropy (S_μν) Field", grid=false, xticks=false, yticks=false)
    heatmap!(p[2], 1:LATTICE_SIZE, 1:LATTICE_SIZE, entropy_history[frame, :, :]', c=:viridis, colorbar=false)
end
gif(anim, "ngft_lensing_simulation.gif", fps=30)
println("Animation saved as ngft_lensing_simulation.gif")
# ... (rest of analysis code) ...

--- NGFT Gravitational Lensing Simulation (V6 - Final Tuning) ---
Phase 1: Running physics calculations...
Phase 1 complete.
Phase 2: Rendering animation...
Animation saved as ngft_lensing_simulation.gif


[ Info: Saved animation to /content/ngft_lensing_simulation.gif
